In [3]:
%pylab inline

rcParams["figure.figsize"] = (12, 9) #<--- large default figures

# Plot text elements
rcParams['axes.labelsize'] = 17
rcParams['axes.titlesize'] = 17
rcParams['xtick.labelsize'] = 15
rcParams['ytick.labelsize'] = 15

Populating the interactive namespace from numpy and matplotlib


# Read tweets appearing on homepage

In [4]:
import tweepy
import pandas as pd

consumer_key = 'JElhoUDXi9BFsfCTVERhc9Vb1'
consumer_secret = 'CPws1P5KCYWbVePYKN0FpBmiSV1hvcow4x4O1lwRS4G1U31Jxl'

access_token ='3773670734-SHThwjiBZVvI34rdjWn7lGgCAM2YHLOL0OPCrrC'
access_token_secret = 'gGHhMkrKCOVQhvMRTOB0asLE7ZFIsNmcjN8s0rAZIqY07'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth)

public_tweets = api.home_timeline()
for (idx, tweet) in enumerate(public_tweets[0:3]): #First 3 tweets in my public feed
    print ('TWEET', idx, ': ', tweet.text)


TWEET 0 :  Step out of the Shadows with Bimodal IT and Private #Cloud https://t.co/hFIgLSW6vf https://t.co/NFot2420qn
TWEET 1 :  Newfound Spider Species Masquerades As a Dried-Up Leaf https://t.co/zhFOP8fUMQ https://t.co/PK605MkJBx
TWEET 2 :  frieze video: We're 25! Dan Fox reflects on how magazine publishing &amp; contemporary art have changed https://t.co/4dbPwUk93k


# Collect election-related tweets using Twitter Search API

In [12]:
# The following code is taken from Alexander Galea's blog:
# https://galeascience.wordpress.com/2016/03/18/collecting-twitter-data-with-python/

import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys


'''
The main() function can be run by executing the command: 
python twitter_search.py
'''

def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''

    consumer_key = 'JElhoUDXi9BFsfCTVERhc9Vb1'
    consumer_secret = 'CPws1P5KCYWbVePYKN0FpBmiSV1hvcow4x4O1lwRS4G1U31Jxl'
    access_token = '3773670734-SHThwjiBZVvI34rdjWn7lGgCAM2YHLOL0OPCrrC'
    access_secret = 'gGHhMkrKCOVQhvMRTOB0asLE7ZFIsNmcjN8s0rAZIqY07'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    api = tweepy.API(auth)
    return api
    # show the tweepy rate limits
    print (api.rate_limit_status())

    
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''

    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
				                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id


def get_tweet_id(api, date='', days_ago=9, query='a'):
    ''' Function that gets the ID of a tweet. This ID can then be
        used as a 'starting point' from which to search. The query is
        required and has been set to a commonly used word by default.
        The variable 'days_ago' has been initialized to the maximum
        amount we are able to search back in time (9).'''

    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id


def write_tweets(tweets, filename):
    ''' Function that appends tweets to a file. '''

    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')


def main():
    ''' This is a script that continuously searches for tweets
        that were created over a given number of days. The search
        dates and search phrase can be changed below. '''

    ''' search variables: '''
    '''
    search_phrases = ['#imwithher', 
                      '#makeamericagreatagain',
                      'hillary',                      # odd dates
                      'trump',
                      '#election2016',
                      'election',
                      '#ivoted',
                      '#voted',
                      'POTUS']
   
    search_phrases = ['#donaldtrump',                 # repeated searches: #imwithher,
                      '#election2016',                # #makeamericagreatagain, #ivoted,
                      '#hillaryclinton',              # #election2016
                      '#imwithher',                   # even dates
                      '#ivoted',
                      '#maga',
                      '#makeamericagreatagain',
                      '#makedonalddrumpfagain',
                      '#neverhillary',
                      '#nevertrump']
    '''

    search_phrases = ['#donaldtrump',               
                      '#election2016',          
                      '#hillaryclinton',             
                      '#imwithher',
                      '#ivoted',                         
                      '#maga',                          
                      '#makeamericagreatagain',         
                      '#makedonalddrumpfagain',
                      '#neverhillary',
                      '#nevertrump',      
                      'hillary',
                      'trump',
                      'election',
                      '#voted',
                      'POTUS']

    
    # Specify search limits: a maximum number of days old (up to about 8) 
    # and a minimum number of days (as low as 0 i.e., “right now”). Collecting tweets over only 
    # one-day intervals is beneficial because each day is exported into its own file.
    
    time_limit = 0.5                           # runtime limit in hours (0.1 hours = 6 minutes)
    max_tweets = 100                           # number of tweets per each search (will be
                                               # iterated) - maximum is 100
    min_days_old, max_days_old = 2, 7          # search limits (8 to 9) e.g., from 7 to 8
                                               # gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '39.8,-95.583068847656,2500km'       # this geocode includes all lower 48 American
                                               # states (and a large portion of Mexico and Canada)
    
    # loop over search items,
    # creating a new file for each
    for search_phrase in search_phrases:

        print('Search phrase =', search_phrase)

        ''' other variables '''
        name = search_phrase.split()[0]
        json_file_root = name + '/'  + name
        os.makedirs(os.path.dirname(json_file_root), exist_ok=True)
        read_IDs = False
        
        # open a file in which to store the tweets
        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                  d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        if os.path.isfile(json_file):
            print('Appending tweets to file named: ',json_file)
            read_IDs = True
        
        # authorize and load the twitter API
        api = load_api()
        
        # set the 'starting point' ID for tweet collection
        if read_IDs:
            # open the json file and get the latest tweet ID
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                print('Searching from the bottom ID in file')
        else:
            # get the ID of a tweet that is min_days_old
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = get_tweet_id(api, days_ago=(min_days_old-1))
        # set the smallest ID to search for
        since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        print('max id (starting point) =', max_id)
        print('since id (ending point) =', since_id)
        


        ''' tweet gathering loop  '''
        start = dt.datetime.now()
        end = start + dt.timedelta(hours=time_limit)
        count, exitcount = 0, 0
        while dt.datetime.now() < end:
            count += 1
            print('count =',count)
            # collect tweets and update max_id
            tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                          max_id=max_id, since_id=since_id,
                                          geocode=USA)
            # write tweets to file in JSON format
            if tweets:
                write_tweets(tweets, json_file)
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrases[-1]:
                        sys.exit('Maximum number of empty tweet strings reached - exiting')
                    else:
                        print('Maximum number of empty tweet strings reached - breaking')
                        break


if __name__ == "__main__":
    main()

Search phrase = #donaldtrump
search limit (start/stop): 2016-11-14 23:59:59
search limit (start/stop): 2016-11-09 23:59:59
max id (starting point) = 798314569517121537
since id (ending point) = 796502630197526528
count = 1
found 100 tweets
count = 2
found 100 tweets
count = 3
found 100 tweets
count = 4
found 100 tweets
count = 5
found 100 tweets
count = 6
found 100 tweets
count = 7
found 100 tweets
count = 8
found 100 tweets
count = 9
found 100 tweets
count = 10
found 100 tweets
count = 11
found 100 tweets
count = 12
found 100 tweets
count = 13
found 100 tweets
count = 14
found 100 tweets
count = 15
found 100 tweets
count = 16
found 100 tweets
count = 17
found 100 tweets
count = 18
found 100 tweets
count = 19
found 100 tweets
count = 20
found 100 tweets
count = 21
found 100 tweets
count = 22
found 100 tweets
count = 23
found 100 tweets
count = 24
found 100 tweets
count = 25
found 100 tweets
count = 26
found 100 tweets
count = 27
found 100 tweets
count = 28
found 100 tweets
count = 29
f

In [6]:
%cd '/Users/evelyn/Workspaces/springboard/capstone/electiontweets'

/Users/evelyn/Workspaces/springboard/capstone/electiontweets
